## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hokitika,NZ,-42.7167,170.9667,61.77,92,99,14.79,overcast clouds
1,1,Karratha,AU,-20.7377,116.8463,90.79,45,4,11.83,clear sky
2,2,Bethel,US,41.3712,-73.4140,53.60,54,1,13.80,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,48.20,71,75,12.66,broken clouds
4,4,Qaqortoq,GL,60.7167,-46.0333,5.00,36,75,8.05,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hokitika,NZ,-42.7167,170.9667,61.77,92,99,14.79,overcast clouds
1,1,Karratha,AU,-20.7377,116.8463,90.79,45,4,11.83,clear sky
6,6,Prado,BR,-17.3411,-39.2208,72.09,90,1,3.96,clear sky
7,7,Kattivakkam,IN,13.2167,80.3167,89.60,55,40,4.61,scattered clouds
10,10,Cabedelo,BR,-6.9811,-34.8339,75.20,100,40,7.78,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                360
City                   360
Country                355
Lat                    360
Lng                    360
Max Temp               360
Humidity               360
Cloudiness             360
Wind Speed             360
Current Description    360
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hokitika,NZ,-42.7167,170.9667,61.77,92,99,14.79,overcast clouds
1,1,Karratha,AU,-20.7377,116.8463,90.79,45,4,11.83,clear sky
6,6,Prado,BR,-17.3411,-39.2208,72.09,90,1,3.96,clear sky
7,7,Kattivakkam,IN,13.2167,80.3167,89.60,55,40,4.61,scattered clouds
10,10,Cabedelo,BR,-6.9811,-34.8339,75.20,100,40,7.78,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
663,663,Vega De Alatorre,MX,20.0333,-96.6333,73.54,81,3,11.70,clear sky
664,664,Zemio,CF,5.0314,25.1361,71.06,90,100,1.21,overcast clouds
665,665,Natchitoches,US,31.7668,-93.1335,71.60,88,90,4.61,overcast clouds
666,666,Ngunguru,NZ,-35.6167,174.5000,68.00,81,99,8.01,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hokitika,NZ,61.77,overcast clouds,-42.7167,170.9667,
1,Karratha,AU,90.79,clear sky,-20.7377,116.8463,
6,Prado,BR,72.09,clear sky,-17.3411,-39.2208,
7,Kattivakkam,IN,89.60,scattered clouds,13.2167,80.3167,
10,Cabedelo,BR,75.20,scattered clouds,-6.9811,-34.8339,
11,Rikitea,PF,79.21,clear sky,-23.1203,-134.9692,
18,Port Elizabeth,ZA,66.20,clear sky,-33.9180,25.5701,
20,Atuona,PF,80.69,light rain,-9.8000,-139.0333,
27,Sao Joao Da Barra,BR,75.07,few clouds,-21.6403,-41.0511,
28,Carnarvon,AU,82.40,broken clouds,-24.8667,113.6333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")            
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hokitika,NZ,61.77,overcast clouds,-42.7167,170.9667,Beachfront Hotel Hokitika
1,Karratha,AU,90.79,clear sky,-20.7377,116.8463,ibis Styles Karratha
6,Prado,BR,72.09,clear sky,-17.3411,-39.2208,INN CORNER RIVER - PRADO - BAHIA - HOUSING AND...
7,Kattivakkam,IN,89.60,scattered clouds,13.2167,80.3167,Raj's villa
10,Cabedelo,BR,75.20,scattered clouds,-6.9811,-34.8339,Hotel Pigalle
...,...,...,...,...,...,...,...
663,Vega De Alatorre,MX,73.54,clear sky,20.0333,-96.6333,El Polo Norte
664,Zemio,CF,71.06,overcast clouds,5.0314,25.1361,OCHA Office
665,Natchitoches,US,71.60,overcast clouds,31.7668,-93.1335,Super 8 by Wyndham Natchitoches
666,Ngunguru,NZ,68.00,overcast clouds,-35.6167,174.5000,Pacific Rendezvous Motel Resort


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))